<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_1000rep_n70_x4_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_1000rep_n70_x4_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...         y_hat         e_hat
rep   index                        ...                            
0.0   0.0    39.820191  33.427731  ... -16339.070967 -4.281215e+04
      1.0    34.678561  39.290675  ... -10020.960755 -1.393475e+06
      2.0    10.411925  35.043939  ...  12485.895904 -4.315830e+03
      3.0    48.573779  19.903448  ... -19664.083191  2.043744e+05
      4.0    38.494309  38.184238  ... -13562.765540 -3.258309e+06
...                ...        ...  ...           ...           ...
999.0 65.0   18.186943  35.462385  ...   2358.761459  4.926084e+04
      66.0   28.716468  39.570157  ...  -4050.097735 -4.864423e+05
      67.0   36.144495  48.232490  ... -17111.428167 -3.133460e+06
      68.0   33.331752  46.245043  ... -14179.384636  1.105660e+06
      69.0   33.916439  26.460723  ...  -9067.927817 -4.777724e+05

[70000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...         y_hat         e_hat
rep   index                        ...                            
0.0   0.0    39.820191  33.427731  ... -16339.070967 -4.281215e+04
      1.0    34.678561  39.290675  ... -10020.960755 -1.393475e+06
      2.0    10.411925  35.043939  ...  12485.895904 -4.315830e+03
      3.0    48.573779  19.903448  ... -19664.083191  2.043744e+05
      4.0    38.494309  38.184238  ... -13562.765540 -3.258309e+06
...                ...        ...  ...           ...           ...
999.0 65.0   18.186943  35.462385  ...   2358.761459  4.926084e+04
      66.0   28.716468  39.570157  ...  -4050.097735 -4.864423e+05
      67.0   36.144495  48.232490  ... -17111.428167 -3.133460e+06
      68.0   33.331752  46.245043  ... -14179.384636  1.105660e+06
      69.0   33.916439  26.460723  ...  -9067.927817 -4.777724e+05

[70000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,39.820191,33.427731,21.895403,-5.960974e+04,-5.915122e+04,-16339.070967,-4.281215e+04
1,0.0,1.0,34.678561,39.290675,45.874624,-1.403933e+06,-1.403496e+06,-10020.960755,-1.393475e+06
2,0.0,2.0,10.411925,35.043939,27.283121,7.998619e+03,8.170066e+03,12485.895904,-4.315830e+03
3,0.0,3.0,48.573779,19.903448,24.298106,1.841754e+05,1.847103e+05,-19664.083191,2.043744e+05
4,0.0,4.0,38.494309,38.184238,44.306878,-3.272344e+06,-3.271872e+06,-13562.765540,-3.258309e+06
...,...,...,...,...,...,...,...,...,...
69995,999.0,65.0,18.186943,35.462385,10.400342,5.138687e+04,5.161960e+04,2358.761459,4.926084e+04
69996,999.0,66.0,28.716468,39.570157,47.736966,-4.908719e+05,-4.904924e+05,-4050.097735,-4.864423e+05
69997,999.0,67.0,36.144495,48.232490,28.667739,-3.151015e+06,-3.150572e+06,-17111.428167,-3.133460e+06
69998,999.0,68.0,33.331752,46.245043,25.013112,1.091071e+06,1.091481e+06,-14179.384636,1.105660e+06


In [6]:
table.set_index(['rep', 'index'], inplace=True)

In [7]:
table

level_0         x1  ...         y_hat         e_hat
rep   index                      ...                            
0.0   0.0          0  39.820191  ... -16339.070967 -4.281215e+04
      1.0          1  34.678561  ... -10020.960755 -1.393475e+06
      2.0          2  10.411925  ...  12485.895904 -4.315830e+03
      3.0          3  48.573779  ... -19664.083191  2.043744e+05
      4.0          4  38.494309  ... -13562.765540 -3.258309e+06
...              ...        ...  ...           ...           ...
999.0 65.0     69995  18.186943  ...   2358.761459  4.926084e+04
      66.0     69996  28.716468  ...  -4050.097735 -4.864423e+05
      67.0     69997  36.144495  ... -17111.428167 -3.133460e+06
      68.0     69998  33.331752  ... -14179.384636  1.105660e+06
      69.0     69999  33.916439  ...  -9067.927817 -4.777724e+05

[70000 rows x 8 columns]

In [8]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index', 'level_0'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid

    SSR1 = model1.ssr

    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid

    SSR0 = model0.ssr

    MSE0 = model0.mse_resid

    df1 = DF1+1
    df0 = DF0+1

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/df1) / (SSR0/df0)


    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.p

In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
5.0,40.0,24.0,143761282912396.44,182186007520451.22,3686186741343.4985,7921130761758.749,2.1121357553024156,0.03547528327000604,0.982262358364997,Reject005=0 : Heteroscedasticity
6.0,28.0,36.0,1910332817520.437,335816510375255.4,70753067315.57175,9594757439293.01,136.7251909119024,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
9.0,34.0,30.0,26640479516938.223,244285995243081.34,807287258089.0371,8423655008382.115,10.392360283885434,1.0210181589087597e-09,0.9999999994894909,Reject005=0 : Heteroscedasticity
12.0,40.0,24.0,69931136629253.484,94768243371758.61,1793106067416.756,4120358407467.7656,2.2586086813322606,0.022007184266745083,0.9889964078666275,Reject005=0 : Heteroscedasticity
17.0,32.0,32.0,63540413702115.266,132986102404323.73,2049690764584.3635,4289874271107.2173,2.092937308022227,0.040359029317053485,0.9798204853414733,Reject005=0 : Heteroscedasticity
...,...,...,...,...,...,...,...,...,...,...
991.0,36.0,28.0,110410233147666.84,156409777052828.84,3154578089933.3384,5792954705660.327,1.8213736086695156,0.09048830217163362,0.9547558489141832,Reject005=1 : Homoscedasticity
995.0,29.0,35.0,10339365188563.41,383284038390482.25,369263042448.69324,11273059952661.242,30.715445043870595,4.6629367034256575e-15,0.9999999999999977,Reject005=0 : Heteroscedasticity
996.0,30.0,34.0,89773982848723.92,291327390444797.7,3095654580990.48,8828102740751.445,2.8633416012898567,0.004420410784195639,0.9977897946079022,Reject005=0 : Heteroscedasticity


In [10]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,38.0,26.0,313094385293482.2,891848328246.8413,8462010413337.356,35673933129.87365,0.0041631875077214045,3.081288973923975e-25,Reject005=0 : Heteroscedasticity
1.0,25.0,39.0,319739627233622.94,127739323202809.03,13322484468067.623,3361561136916.0273,0.25609644399951775,0.0001437035768417449,Reject005=0 : Heteroscedasticity
2.0,30.0,34.0,316648752494218.75,6324684938403.779,10918922499800.646,191657119345.56906,0.017623958134864236,3.7405972893185355e-21,Reject005=0 : Heteroscedasticity
3.0,31.0,33.0,292307041890848.3,4022137631661.2485,9743568063028.277,125691800989.41402,0.012926037259142627,7.357663850703836e-23,Reject005=0 : Heteroscedasticity
4.0,27.0,37.0,370534126228733.94,4537871507325.815,14251312547258.998,126051986314.60599,0.008936881960893773,1.0186873932283317e-27,Reject005=0 : Heteroscedasticity
...,...,...,...,...,...,...,...,...,...
995.0,29.0,35.0,10339365188563.41,383284038390482.25,369263042448.69324,11273059952661.242,30.715445043870595,4.6629367034256575e-15,Reject005=0 : Heteroscedasticity
996.0,30.0,34.0,89773982848723.92,291327390444797.7,3095654580990.48,8828102740751.445,2.8633416012898567,0.004420410784195639,Reject005=0 : Heteroscedasticity
997.0,38.0,26.0,181787005610401.9,88506100326753.31,4913162313794.646,3540244013070.1323,0.7115748965333638,0.366313959679013,Reject005=1 : Homoscedasticity


In [11]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    715
Reject005=1 : Homoscedasticity      285
Name: Result_test, dtype: int64